<a href="https://colab.research.google.com/github/nbisso/TP2-Orga-7506/blob/main/TP_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://realpython.com/train-test-split-python-data/

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import f1_score

In [ ]:
df_train_labels = pd.read_csv('/content/drive/MyDrive/datasets_tp2/train_labels.csv')

In [ ]:
df_train_values = pd.read_csv('/content/drive/MyDrive/datasets_tp2/train_values.csv', dtype = {'land_surface_condition':'category', 'foundation_type':'category'} )

In [ ]:
df_train_values.dtypes

In [ ]:
df_train_labels.head()

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [ ]:
df_train_values.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,1,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,0,1,0,0,0,0,1,1,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,1,0,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#df_train_values = df_train_values.set_index('building_id')

In [ ]:
columnas = ['foundation_type', 
                     'area_percentage', 
                     'height_percentage',
                     'count_floors_pre_eq',
                     'land_surface_condition',
                     'has_superstructure_cement_mortar_stone', 'damage_grade']

In [ ]:
df_train_values['damage_grade'] = df_train_labels['damage_grade']

In [ ]:
df = df_train_values[columnas]
df['damage_grade'] = df['damage_grade'] -1
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,foundation_type,area_percentage,height_percentage,count_floors_pre_eq,land_surface_condition,has_superstructure_cement_mortar_stone,damage_grade
0,r,6,5,2,t,0,2
1,r,8,7,2,o,0,1
2,r,5,5,2,t,0,2
3,r,6,5,2,t,0,1
4,r,8,9,3,t,0,2


In [ ]:
X,y = df.iloc[:, :-1], df.iloc[:, -1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [ ]:
X_test

,foundation_type,area_percentage,height_percentage,count_floors_pre_eq,land_surface_condition,has_superstructure_cement_mortar_stone
242724,r,6,3,1,t,0
197792,r,3,5,2,t,0
50830,r,6,6,3,t,0
229772,r,14,4,2,t,0
90617,r,6,5,2,t,0
...,...,...,...,...,...,...
157672,r,3,5,2,t,0
253672,r,11,6,3,t,0
176899,r,8,5,2,t,0
160311,r,7,6,3,t,0


In [ ]:
lgb_train = lgb.Dataset(X_train, label = y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference= lgb_train)
lgb_eval.data

,foundation_type,area_percentage,height_percentage,count_floors_pre_eq,land_surface_condition,has_superstructure_cement_mortar_stone
242724,r,6,3,1,t,0
197792,r,3,5,2,t,0
50830,r,6,6,3,t,0
229772,r,14,4,2,t,0
90617,r,6,5,2,t,0
...,...,...,...,...,...,...
157672,r,3,5,2,t,0
253672,r,11,6,3,t,0
176899,r,8,5,2,t,0
160311,r,7,6,3,t,0


In [ ]:
y_test

242724    1
197792    1
50830     2
229772    1
90617     2
         ..
157672    1
253672    2
176899    2
160311    2
139061    2
Name: damage_grade, Length: 78181, dtype: int64

In [ ]:
params = {
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_classes': 3,
    'num_leaves': 40,
    'learning_rate': 0.05,
    'max_depth': 8
}

In [ ]:
#https://neptune.ai/blog/lightgbm-parameters-guide
model = lgb.train(params,
                lgb_train,
                num_boost_round=200,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)

/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	valid_0's multi_logloss: 0.904021
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's multi_logloss: 0.897968
[3]	valid_0's multi_logloss: 0.892555
[4]	valid_0's multi_logloss: 0.887667
[5]	valid_0's multi_logloss: 0.883227
[6]	valid_0's multi_logloss: 0.87917
[7]	valid_0's multi_logloss: 0.875453
[8]	valid_0's multi_logloss: 0.872029
[9]	valid_0's multi_logloss: 0.86887
[10]	valid_0's multi_logloss: 0.865945
[11]	valid_0's multi_logloss: 0.863232
[12]	valid_0's multi_logloss: 0.86071
[13]	valid_0's multi_logloss: 0.858363
[14]	valid_0's multi_logloss: 0.856172
[15]	valid_0's multi_logloss: 0.854129
[16]	valid_0's multi_logloss: 0.852219
[17]	valid_0's multi_logloss: 0.850434
[18]	valid_0's multi_logloss: 0.848764
[19]	valid_0's multi_logloss: 0.847187
[20]	valid_0's multi_logloss: 0.845711
[21]	valid_0's multi_logloss: 0.84434
[22]	valid_0's multi_logloss: 0.843044
[23]	valid_0's multi_logloss: 0.841826
[24]	valid_0's multi_logloss: 0.840695
[25]	valid_0's 

In [ ]:
pred = model.predict(X_test)
pred

array([[0.11780734, 0.49890757, 0.3832851 ],
       [0.03826826, 0.51871659, 0.44301515],
       [0.01660096, 0.45823029, 0.52516875],
       ...,
       [0.04664684, 0.62060258, 0.33275058],
       [0.01652425, 0.4897778 , 0.49369795],
       [0.04574817, 0.60027952, 0.35397231]])

In [ ]:
labels_pred = [np.argmax(v) for v in pred]
labels_pred

In [ ]:
error = f1_score(y_test, labels_pred, average ='micro')
error

0.5851933334186056

In [ ]:
df = pd.DataFrame(data = {'building_id': [], 'damage_grade': []})
